In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys, os
sys.path.append('..')

import numpy as np
import librosa as lr
import torch
import IPython.display as ipd
import matplotlib.pyplot as plt
import pytorch_lightning as pl

from scipy.signal.windows import hann
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor, EarlyStopping

from datasets.nsynth_datamodule import NsynthDataModule
from models.cvae_resnet import CvaeResnet
from models.cvae_inception import CvaeInception

pl.seed_everything(42)

Global seed set to 42


42

In [12]:
torch.cuda.set_device(1)
print(torch.cuda.current_device())


1


In [32]:
### CONFIGS

model_type = 'incept'
num_workers = 8
batch_size = 16
max_epochs = 20000
patience = 750
gpus = '1'

ds_configs = {
    'dataset_path': '/data/riccardo_datasets',
    'feature': 'spec',
    'feature_params': {
        'win_length': 256,
        'hop_length': 64,
        'window': hann(256)
    },
    'n_fft': 510,
    'ds_kwargs': {
        #'pitches': [60],
        #'instrument_families': [0],
        'sr': 16000,
        'duration': 1.02
    }
}

m_configs_incept = {
    'lr': 1e-3,
    'lr_scheduler': {
        'factor': 10**(-1/3), 
        'patience': 200,
        'cooldown': 100,
        'min_lr': 1e-5
    },
    'c_labels': ['pitch'],
    'kl_coeff': 1e-4,
    'db_coeff': 5e-3,
    'latent_size': 32,
    'channel_size': 2,
    'use_inception': True,
    'repeat_per_block': 1,
    'dense_size': 256,
}


In [25]:
# init model
model = CvaeInception(m_configs_incept)

In [26]:
# logger
log_name = '{}_{}'.format(CvaeInception.model_name, 'test_pitches')
logger = TensorBoardLogger('logs', name=log_name)

In [36]:
# init data loader
dm = NsynthDataModule(ds_configs, num_workers=num_workers, batch_size=batch_size)

In [ ]:
# callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=patience)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

# train!
trainer = pl.Trainer(
#    weights_summary='full',
#    overfit_batches=1,
#    terminate_on_nan=False,
#    gradient_clip_val=0.5,
    max_epochs=max_epochs,
    callbacks=[early_stop, lr_monitor],
    logger=logger,
    gpus=gpus)
trainer.fit(model=model, datamodule=dm)

In [ ]:
trainer.fit(model=model, datamodule=dm)


  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Encoder | 3.8 M 
1 | fc_mu     | Linear  | 8.3 K 
2 | fc_logvar | Linear  | 8.3 K 
3 | fc_rep    | Linear  | 8.7 K 
4 | decoder   | Decoder | 2.2 M 
--------------------------------------
5.9 M     Trainable params
0         Non-trainable params
5.9 M     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 791it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# callbacks
lr_monitor = LearningRateMonitor(logging_interval='epoch')

# train!
trainer = pl.Trainer(
    max_epochs=20000,
    overfit_batches=1,
    callbacks=[lr_monitor],
    gpus=gpus)
trainer.fit(model=model, datamodule=dm)